In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
train = '/content/drive/My Drive/X-RayProject/train'
validation = '/content/drive/My Drive/X-RayProject/validation'
test = '/content/drive/My Drive/X-RayProject/test' 

In [ ]:
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Dropout, Input, BatchNormalization
from keras import backend as K
from keras.callbacks import EarlyStopping
import keras
from time import time

import os
import numpy as np 

In [ ]:
#-------PARAMETERS--------

height=224 #height equals to width but if you need to change some of this parameters you can
width=224
nChannels=1
nClasses=3
batch_size=16
epochs=200

In [ ]:
#------------DATA AUGMENTATION-----------
datagen = ImageDataGenerator (
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
valdatagen = ImageDataGenerator (
    rescale=1./255, 
)
train_generator = datagen.flow_from_directory(  
        train,                                                          
        target_size=(height, width),                                            
        batch_size=batch_size,                                                                                                        
        class_mode='categorical',
        color_mode='grayscale')

validation_generator = valdatagen.flow_from_directory(  
        validation,                                                          
        target_size=(height, width),                                            
        batch_size=batch_size,                                                                                                        
        class_mode='categorical',
        color_mode='grayscale')

Found 6651 images belonging to 4 classes.
Found 1459 images belonging to 4 classes.


In [ ]:
# MODEL --------------------------------------------------

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(width, height, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu')) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

# TRAINING --------------------------------------------------

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

model.fit(
        train_generator,
        epochs=epochs,
        validation_data = validation_generator,
        callbacks = [es]
)

# SAVING --------------------------------------------------

model.save("x-ray.h1")

Epoch 1/200
416/416 [==============================] - 2834s 7s/step - loss: 0.8222 - accuracy: 0.6473 - val_loss: 0.8211 - val_accuracy: 0.6875
Epoch 2/200
416/416 [==============================] - 255s 613ms/step - loss: 0.5838 - accuracy: 0.7673 - val_loss: 0.7247 - val_accuracy: 0.7176
Epoch 3/200
416/416 [==============================] - 254s 610ms/step - loss: 0.4826 - accuracy: 0.8155 - val_loss: 0.4971 - val_accuracy: 0.8273
Epoch 4/200
416/416 [==============================] - 255s 613ms/step - loss: 0.4303 - accuracy: 0.8444 - val_loss: 0.6069 - val_accuracy: 0.7656
Epoch 5/200
416/416 [==============================] - 259s 622ms/step - loss: 0.4017 - accuracy: 0.8540 - val_loss: 0.5701 - val_accuracy: 0.7649
Epoch 6/200
416/416 [==============================] - 257s 618ms/step - loss: 0.3676 - accuracy: 0.8618 - val_loss: 0.5516 - val_accuracy: 0.8088
Epoch 7/200
416/416 [==============================] - 255s 613ms/step - loss: 0.3419 - accuracy: 0.8714 - val_loss: 0.5